In [1]:
import pandas as pd
import os
import json

In [82]:
df = pd.read_json("./original_json/11번수정.json")

In [83]:
df.head()

,file_name,language,duration,segments
0,11번수정.mp3,ko,281.22,"{'start': 0.0, 'end': 5.5, 'text': '전라도 광주 출신이..."
1,11번수정.mp3,ko,281.22,"{'start': 5.5, 'end': 10.5, 'text': '이걸 왜 여쭤봤냐..."
2,11번수정.mp3,ko,281.22,"{'start': 10.5, 'end': 13.5, 'text': '금융범죄 사기단..."
3,11번수정.mp3,ko,281.22,"{'start': 13.5, 'end': 19.2, 'text': '검거 현장에서 ..."
4,11번수정.mp3,ko,281.22,"{'start': 19.2, 'end': 22.0, 'text': '명의로 개설된 ..."


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   file_name  58 non-null     object 
 1   language   58 non-null     object 
 2   duration   58 non-null     float64
 3   segments   58 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.9+ KB


# 데이터 전처리

- 파일명, 언어컬럼 삭제
- segments 컬럼에서 start, end, text분할하여 새로운 컬럼을 생성

In [18]:
df = pd.read_json("./original_json/5번수정.json")

In [19]:
df.head()

,file_name,language,duration,segments
0,5번수정.mp3,ko,151.26,"{'start': 0.0, 'end': 9.68, 'text': '계좌가 아니기 때..."
1,5번수정.mp3,ko,151.26,"{'start': 9.68, 'end': 20.08, 'text': '예 알겠습니다..."
2,5번수정.mp3,ko,151.26,"{'start': 20.08, 'end': 27.16, 'text': '그렇기 때문..."
3,5번수정.mp3,ko,151.26,"{'start': 27.16, 'end': 32.16, 'text': '예 그러시면..."
4,5번수정.mp3,ko,151.26,"{'start': 32.16, 'end': 43.76, 'text': 'OOO씨한테..."


In [22]:
# 1차 전처리 및 데이터프레임화 함수 생성
def pre1(df):
    '''
    JSON 1개를 DataFrame([data]) 형태로 받아
    발화 단위 DataFrame으로 반환
    '''
    
    # 필요없는 컬럼 삭제
    df = df.drop(["file_name", "language"], axis = 1)

    segments = df.loc[0, "segments"]
    
    # segments 컬럼에서 발화 시작,끝, 텍스트 추출 및 분할
    start, end, text = [], [], []
    
    for seg in segments:
        start.append(seg["start"])
        end.append(seg["end"])
        text.append(seg["text"])
    
    result = pd.DataFrame({
        "총 발화시간": [df.loc[0, "duration"]] * len(start),
        "발화_시작시간": start,
        "발화_종료시간": end,
        "발화_내용": text
    })

    # 발화시간 컬럼 추가 (파생변수 생성1)
    result["발화_지속시간"] = result["발화_종료시간"] - result["발화_시작시간"]

    # 발화길이 컬럼 추가(파생변수 생성2)
    result["발화_길이"] = result["발화_내용"].str.len()

    # 이전 침묵 시간 컬럼 추가 (파생변수 생성3)
    silence = []
    for i in range(len(result)):
        if i == len(result) - 1:
            silence.append(0)
        else:
            silence.append(result["발화_시작시간"][i+1] - result["발화_종료시간"][i])
    
    result["이전_침묵시간"] = silence

    # 통화상대위치 컬럼 추가 (파생변수 생성4)
    result["통화_상대위치"] = result["발화_시작시간"] / result["총 발화시간"]

    return result

In [23]:
# 결과를 담을 딕셔너리
json_dir = "./test_json"  # JSON 파일들이 있는 폴더
dfs = {}

for file in os.listdir(json_dir):
    if file.endswith(".json"):
        # 1. JSON 로드
        with open(os.path.join(json_dir, file), "r", encoding="utf-8") as f:
            data = json.load(f)

        # 2. JSON → DataFrame
        df_raw = pd.DataFrame([data])

        # 3. 전처리 함수 실행
        dfs[file] = pre1(df_raw)

In [24]:
dfs[file]

,총 발화시간,발화_시작시간,발화_종료시간,발화_내용,발화_지속시간,발화_길이,이전_침묵시간,통화_상대위치
0,306.53,0.00,4.24,그리고 조사 과정에서 사건이나 본인의 정보가 추가적으로 유출되는 경우가 많으니까,4.24,44,0.0,0.000000
1,306.53,4.24,8.84,제대로 본인이 알고 계시는 제3자의 인물이 없는 곳에서 통화 좀 부탁드리겠는데요.,4.60,45,0.0,0.013832
2,306.53,8.84,9.64,아 네.,0.80,4,0.0,0.028839
3,306.53,9.64,14.94,본인 현재 뭐 차량에서 하차하신 것 같은데 지금 직장을 복귀하시는 건가요?,5.30,41,0.0,0.031449
4,306.53,14.94,16.90,아니요. 지금 직장을 안 다녀요.,1.96,18,0.0,0.048739
...,...,...,...,...,...,...,...,...
72,306.53,294.50,296.50,그리고 조사 과정에서 사건이나,2.00,16,0.0,0.960754
73,306.53,296.50,299.00,본인의 정보가 추가적으로 유출되는 경우가 많으니까,2.50,27,0.0,0.967279
74,306.53,299.00,302.00,때때로 인물이 없는 곳에서 통화 좀 부탁드리겠는데요.,3.00,29,0.0,0.975435
75,306.53,302.00,304.50,본인 현직으로 근무하고 계신다고 하셨잖아요.,2.50,24,0.0,0.985222


In [31]:
type(dfs)

dict

In [32]:
dfs.keys()

dict_keys(['2번_3차례 신고된 여성 전화금융사기범 (음성_2).json', '3차-(1).16차례 신고된 남성 전화금융사기범 음성.json', '3차-(2).6차례 신고된 남성 전화금융사기범 음성.json'])

In [41]:
dfs[list(dfs.keys())[2]]

,총 발화시간,발화_시작시간,발화_종료시간,발화_내용,발화_지속시간,발화_길이,이전_침묵시간,통화_상대위치
0,306.53,0.00,4.24,그리고 조사 과정에서 사건이나 본인의 정보가 추가적으로 유출되는 경우가 많으니까,4.24,44,0.0,0.000000
1,306.53,4.24,8.84,제대로 본인이 알고 계시는 제3자의 인물이 없는 곳에서 통화 좀 부탁드리겠는데요.,4.60,45,0.0,0.013832
2,306.53,8.84,9.64,아 네.,0.80,4,0.0,0.028839
3,306.53,9.64,14.94,본인 현재 뭐 차량에서 하차하신 것 같은데 지금 직장을 복귀하시는 건가요?,5.30,41,0.0,0.031449
4,306.53,14.94,16.90,아니요. 지금 직장을 안 다녀요.,1.96,18,0.0,0.048739
...,...,...,...,...,...,...,...,...
72,306.53,294.50,296.50,그리고 조사 과정에서 사건이나,2.00,16,0.0,0.960754
73,306.53,296.50,299.00,본인의 정보가 추가적으로 유출되는 경우가 많으니까,2.50,27,0.0,0.967279
74,306.53,299.00,302.00,때때로 인물이 없는 곳에서 통화 좀 부탁드리겠는데요.,3.00,29,0.0,0.975435
75,306.53,302.00,304.50,본인 현직으로 근무하고 계신다고 하셨잖아요.,2.50,24,0.0,0.985222
